**Student's name**

Mathieu Ruch

An Bui Duc Khanh

This aim of this course is to review the evolution of image processing tools from hand-crafted methods to deep learning algorithms. The semester is split into four labs :

* **Lab 1** : Introduction to Image Processing Using Hand-Crafted Features
* **Lab 2** : Object detection
* **Lab 3** : Object tracking
* **Lab 4** : Introduction to Deep Learning for image classification and generative model

Let's start with the second chapter of this course!

# Chapter 2 : Object Detection and Recognition
(*100 points*)

In [ ]:
# !pip install scikit-learn imageio

import cv2 as cv
import numpy as np
import os
import math
import tarfile
from utils import *
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
import imageio
import matplotlib.pyplot as plt
%matplotlib inline

## 2.1 Template Matching
(*40 points*)

In this warm-up section, we will address the problem of detection and recognition using Template Matching. 

Template matching is a 'brute-force' algorithm for object recognition. The most basic method of template matching is to directly compare the grayscale images, without using edge detection. For example, if you were trying to detect, let's say a football, you will need to create a base template of the object. During the operation, the template matching algorithm would analyze the current image to find areas which are similar to the template. This basic approach is quite limited. For one thing, it is not robust to inconsistent changes in brightness within the image. If the template image has strong features, a feature-based approach may be considered; the approach may prove further useful if the match in the search image might be transformed in some fashion. For templates without strong features, or for when the bulk of the template image constitutes the matching image, a template-based approach may be effective. 

In the naive approach, the difference between the template and the matching area is computed pixel by pixel and used to calculate the overall error. It is possible to reduce the number of sampling points by reducing the resolution of the search and template images by the some factor and performing the operation on the resultant downsized images (multiresolution, or Pyramid (image processing)), providing a search window of data points within the search image so that the template does not have to search every viable data point or a combination of both.

![Template Matching Sample](../data/templateMatch.jpeg)

Template matching example. Left: Template image. Right: Input image with the resulting image highlighted.  

### 2.1.1 Objectives

In this section, we will explore the advantages and disadvantages of template matching method. However, in contrast with the previous Chapter, we will leave most of the implementation for the reader (i.e. you).

The following section will introduce to the most common metrics used for the matching distance and how to are used in OpenCV. Your task will be to: 
 * Implement each metric *by hand* 
 * Compare the accuracy against the OpenCV method 
 * Analyse and report your observations for each metric in 3 exercises and one mini-challenge.

As the final exercise, you will be given a set of more "challenging" data examples where using what you *learned before*, you will be asked to detect several objects in the scene. Your resulting algorithm __should have the given set of inputs and outputs__.

### 2.1.2 Distance, Minimums and Maximums

The two (and pretty much only) important parts of the Naive Template Matching algorithm is the *distance transform*, i.e. the metric to know if we found a match or not, and the global minima detection. 

For an  Input image $I$ if size $W\times H$, a template Image $T$ of size $w\times h$; ($w<W, h<H$), the distance methods implemented in OpenCv are the following: 

* Mean Squared Difference Method=CV_TM_SQDIFF

\begin{equation*}
R(x,y)= \sum _{x',y'} (T(x',y')-I(x+x',y+y'))^2 
\end{equation*}

* Normalized Mean Squared Difference Method=CV_TM_SQDIFF_NORMED
\begin{equation*}
        R(x,y)= \frac{\sum_{x',y'} (T(x',y')-I(x+x',y+y'))^2}{\sqrt{\sum_{x',y'}T(x',y')^2 \cdot \sum_{x',y'} I(x+x',y+y')^2}}
\end{equation*}

* Cross Correlation Method=CV_TM_CCORR
\begin{equation*}
        R(x,y)= \sum _{x',y'} (T(x',y') \cdot I(x+x',y+y'))
\end{equation*}

* Normalized Cross Correlation method=CV_TM_CCORR_NORMED
\begin{equation*}
        R(x,y)= \frac{\sum_{x',y'} (T(x',y') \cdot I(x+x',y+y'))}{\sqrt{\sum_{x',y'}T(x',y')^2 \cdot \sum_{x',y'} I(x+x',y+y')^2}}
\end{equation*}

* Correlation Coefficient Method=CV_TM_CCOEFF
\begin{equation*}
        R(x,y)= \sum _{x',y'} (T'(x',y') \cdot I'(x+x',y+y'))
\end{equation*}

where

\begin{equation*}
\begin{array}{l} T'(x',y')=T(x',y') - 1/(w \cdot h) \cdot \sum _{x'',y''} T(x'',y'') \\ I'(x+x',y+y')=I(x+x',y+y') - 1/(w \cdot h) \cdot \sum _{x'',y''} I(x+x'',y+y'') 
\end{array}
\end{equation*}

* Normalized Correlation Coefficient=CV_TM_CCOEFF_NORMED
\begin{equation*}
        R(x,y)= \frac{ \sum_{x',y'} (T'(x',y') \cdot I'(x+x',y+y')) }{ \sqrt{\sum_{x',y'}T'(x',y')^2 \cdot \sum_{x',y'} I'(x+x',y+y')^2} }
\end{equation*}


Please notice that the dimensions of the output image, $R$, will depend on how you handle the edges. The easiest way is to ser the return an output image of size $(W-w+1, H-h+1)$.

After the function finishes the comparison, the resulting image will contain an image map with the obtained values. In OpenCV, the best matches can be found as global minimums or maximums (depending which matric you used) using the `minMaxLoc()` function. 

**QUESTION** (/3)

For each function listed above, indicate if the best matching position is located either in the local minimums or in the maximums. 


**YOUR ANSWER**

According to the openCV documentation, the best matching position is located in a local minimum for methods using TM_SQDIFF and in a local maximum for methods using TM_CCORR or TM_CCOEFF. So we have that:

The best matching position is located in a minimum for:
- Mean Squared Difference Method (CV_TM_SQDIFF)
- Normalized Mean Squared Difference Method (CV_TM_SQDIFF_NORMED)

The best matching position is located in a maximum for:
- Cross Correlation Method (CV_TM_CCORR)
- Normalized Cross Correlation method (CV_TM_CCORR_NORMED)
- Correlation Coefficient Method (CV_TM_CCOEFF)
- Normalized Correlation Coefficient(CV_TM_CCOEFF_NORMED)

**QUESTION** (/3)

Based on the template  `my_distance_fn()` in the next, implement the functions `sqdiff()`, `sqdiff_normed()`, `ccorr()`, `ccorr_normed()`, `ccoeff()` and `ccoeff_normed()` that takes as input a region of an image, and the template and returns the corresponding distance.

In [ ]:
def my_distance_fn(image_patch, template):
    """
    Calculate the distance between image_patch and template
    :image_patch:       Input region of interest of the image.
    :template:          The Template Image.
    :return:            The dummy distance.
    """
    dummy_distance = np.sum(image_patch.astype(np.float32) - template.astype(np.float32))
    return  dummy_distance



# YOUR CODE HERE
def sqdiff(image_patch, template):
    """
    Calculate the Sum of Squared Differences (SSD) between image_patch and template
    :image_patch:       Input region of interest of the image.
    :template:          The Template Image.
    :return:            The distance between image_patch and template.
    """
    diff = image_patch.astype(np.float32) - template.astype(np.float32)
    ssd = np.sum(np.square(diff))
    return ssd

def sqdiff_normed(image_patch, template):
    """
    Calculate the Normalized Sum of Squared Differences (NSSD) between image_patch and template
    :image_patch:       Input region of interest of the image.
    :template:          The Template Image.
    :return:            The distance between image_patch and template.
    """
    diff = image_patch.astype(np.float32) - template.astype(np.float32)
    numerator = np.sum(np.square(diff))
    
    denominator = np.sqrt(np.sum(image_patch.astype(np.float32)**2) * np.sum(template.astype(np.float32)**2))
    sqdiff_normed = numerator / denominator if denominator != 0 else np.inf
    return sqdiff_normed
    


def ccorr(image_patch, template):
    """
    Calculate the cross-correlation distance between image_patch and template
    :image_patch:       Input region of interest of the image.
    :template:          The Template Image.
    :return:            The cross-correlation distance.
    """
    # Compute cross-correlation distance
    ccorr = np.sum(image_patch.astype(np.float32) * template.astype(np.float32))
    return ccorr


def ccorr_normed(image_patch, template):
    """
    Calculate the normalized cross-correlation distance between image_patch and template
    :image_patch:       Input region of interest of the image.
    :template:          The Template Image.
    :return:            The normalized cross-correlation distance.
    """
    # Compute numerator and denominator of normalized cross-correlation distance
    numerator = np.sum(image_patch.astype(np.float32) * template.astype(np.float32))
    denominator = np.sqrt(np.sum(image_patch.astype(np.float32)**2) * np.sum(template.astype(np.float32)**2))
    
    # Compute normalized cross-correlation distance
    ccorr_normed = numerator / denominator if denominator != 0 else 0
    return ccorr_normed


def ccoeff(image_patch, template):
    """
    Calculates the cross-correlation between image_patch and template.
    """
    sum_T = np.sum(template)
    sum_I = np.sum(image_patch)
    corr = np.sum((image_patch - (sum_I / image_patch.size)) * (template - (sum_T / template.size)))
    return corr

def ccoeff_normed(image_patch, template):
    """
    Calculates the normalized cross-correlation between image_patch and template.
    """
    sum_T = np.sum(template)
    sum_I = np.sum(image_patch)
    numerator = np.sum((image_patch - (sum_I / image_patch.size)) * (template - (sum_T / template.size)))
    denominator = np.sqrt(np.sum(np.square(image_patch - (sum_I / image_patch.size))) * np.sum(np.square(template - (sum_T / template.size))))
    ccoeff_normed = numerator / denominator if denominator != 0 else 0
    return ccoeff_normed
   


The distance functions returns a single value, but we would like the distance between the image and the template at each pixel of the image. To do that, we use a sliding window approach which consists in selecting a region of interest in the image, calculate the distance between this region of interest and the template, and move to the next region of interest. In our case, the regions of interest are all regions of the same size of the template. We start at the pixel (0, 0) and move one pixel by one pixel.

**QUESTION** (/3)

In the next cell, implement the function `template_matching_process()` that takes as input an image, the template, and the distance function and returns the corresponding distance map between the image and the template.

In [ ]:
def template_matching_process(image, template, dist_fn):
    """
    Given an input image, iterates over the image and computes the distance w/r
    the template, using a given distance function. 

    :input_image:       Input image. :) 
    :template:          The Template Image.
    :distance_function: Function used to compute the distance. The function should receive a image patch 
                        and a template as inputs.
    :return:            The distance map.
    """
    height, width = image.shape[:2]
    t_height, t_width = template.shape[:2]
    dist_map = np.zeros((height - t_height + 1, width - t_width + 1), dtype=np.float32)
    
    for y in range(height - t_height + 1):
        for x in range(width - t_width + 1):
            image_patch = image[y:y+t_height, x:x+t_width]
            dist_map[y, x] = dist_fn(image_patch, template)
            

            
    if dist_fn.__name__ == 'sqdiff_normed':

        dist_map[dist_map > 1] = 1


    
    return dist_map

In [ ]:
### Test cell
img_gray = cv.imread('../data/space-invaders_1.jpg', 0)
template = cv.imread('../data/template_0.png', 0)

# calculate distance
dist_map = template_matching_process(img_gray, template, sqdiff)

# display
plt.figure(figsize=(12.8, 8.2))
plt.subplot(131)
plt.imshow(img_gray, cmap='gray')
plt.axis('off')
plt.title('Image')

plt.subplot(132)
plt.imshow(template, cmap='gray')
plt.axis('off')
plt.title('Template')

plt.subplot(133)
plt.imshow(dist_map, cmap='gray')
plt.axis('off')
plt.title('Distance Map')

plt.show()

The next cell tests your implementation with the distance function `sqdiff_normed()`.

In [ ]:
# load image
img_gray = cv.imread('../data/space-invaders_1.jpg', 0)
template = cv.imread('../data/template_0.png', 0)

# calculate distance
dist_map = template_matching_process(img_gray, template, sqdiff_normed)

# display
plt.figure(figsize=(12.8, 8.2))
plt.subplot(131)
plt.imshow(img_gray, cmap='gray')
plt.axis('off')
plt.title('Image')

plt.subplot(132)
plt.imshow(template, cmap='gray')
plt.axis('off')
plt.title('Template')

plt.subplot(133)
plt.imshow(dist_map, cmap='gray')
plt.axis('off')
plt.title('Distance Map')

plt.show()

**QUESTION**  (/3)

Check your outputs by using the build-in functions in OpenCV to compute the maps for each of the methods implemented. Show in each cell: your map and the OpenCV map.

In [ ]:
# Example
res = cv.matchTemplate(img_gray, template, cv.TM_SQDIFF_NORMED)
# Display two example maps
fig, ax = plt.subplots(1, 2, figsize=(15, 7),squeeze=False)
display_image(dist_map, axes=ax[0][0])
ax[0][0].set_title("My Distance Map")
display_image(res, axes=ax[0][1])
ax[0][1].set_title("OpenCv Distance Map")

In [ ]:
# sqdiff()
# YOUR CODE HERE

res = cv.matchTemplate(img_gray, template, cv.TM_SQDIFF)
dist_map = template_matching_process(img_gray, template, sqdiff)
# Display two example maps
fig, ax = plt.subplots(1, 2, figsize=(15, 7),squeeze=False)
display_image(dist_map, axes=ax[0][0])
ax[0][0].set_title("My Distance Map")
display_image(res, axes=ax[0][1])
ax[0][1].set_title("OpenCv Distance Map")

In [ ]:
#sqdiff_normed()
# YOUR CODE HERE

res = cv.matchTemplate(img_gray, template, cv.TM_SQDIFF_NORMED)
dist_map = template_matching_process(img_gray, template, sqdiff_normed)

# Display two example maps
fig, ax = plt.subplots(1, 2, figsize=(15, 7),squeeze=False)
display_image(dist_map, axes=ax[0][0])
ax[0][0].set_title("My Distance Map")
display_image(res, axes=ax[0][1])
ax[0][1].set_title("OpenCv Distance Map")

In [ ]:
#ccorr()
# YOUR CODE HERE

res = cv.matchTemplate(img_gray, template, cv.TM_CCORR)
dist_map = template_matching_process(img_gray, template, ccorr)

# Display two example maps
fig, ax = plt.subplots(1, 2, figsize=(15, 7),squeeze=False)
display_image(dist_map, axes=ax[0][0])
ax[0][0].set_title("My Distance Map")
display_image(res, axes=ax[0][1])
ax[0][1].set_title("OpenCv Distance Map")

In [ ]:
#ccorr_normed()
# YOUR CODE HERE

res = cv.matchTemplate(img_gray, template, cv.TM_CCORR_NORMED)
dist_map = template_matching_process(img_gray, template, ccorr_normed)

# Display two example maps
fig, ax = plt.subplots(1, 2, figsize=(15, 7),squeeze=False)
display_image(dist_map, axes=ax[0][0])
ax[0][0].set_title("My Distance Map")
display_image(res, axes=ax[0][1])
ax[0][1].set_title("OpenCv Distance Map")

In [ ]:
#ccoeff()
# YOUR CODE HERE

res = cv.matchTemplate(img_gray, template, cv.TM_CCOEFF)
dist_map = template_matching_process(img_gray, template, ccoeff)

# Display two example maps
fig, ax = plt.subplots(1, 2, figsize=(15, 7),squeeze=False)
display_image(dist_map, axes=ax[0][0])
ax[0][0].set_title("My Distance Map")
display_image(res, axes=ax[0][1])
ax[0][1].set_title("OpenCv Distance Map")

In [ ]:
#ccoeff_normed()
# YOUR CODE HERE

res = cv.matchTemplate(img_gray, template, cv.TM_CCOEFF_NORMED)
dist_map = template_matching_process(img_gray, template, ccoeff_normed)

# Display two example maps
fig, ax = plt.subplots(1, 2, figsize=(15, 7),squeeze=False)
display_image(dist_map, axes=ax[0][0])
ax[0][0].set_title("My Distance Map")
display_image(res, axes=ax[0][1])
ax[0][1].set_title("OpenCv Distance Map")

**QUESTION** (/2)

Comment the differences between the OpenCV function and yours (speed, precision, etc).

The openCV implementation is more precise than the one we implemented. Manually checking the minimum values in the OpenCV map, we see that it holds values as small as 1e-7, while our minimums are rounded down to 0. Moreover, the OpenCV implementation is faster than ours due to the use of optimized libraries and matrix manipulation functions, while we use for loops to iterate over the image and template matrices.

### 2.1.3 Finding Local Minimums / Maximums

As explained before, in order to find the location of our possible object we need to find the minimum or maximum point in our resulting distance map. The  OpenCV function `cv.minMaxLoc()` can be used to find the local minimum and maximum of the single-channel array (1D or 2D) [[doc](https://docs.opencv.org/4.0.0/d2/de8/group__core__array.html#gab473bf2eb6d14ff97e89b355dac20707)]. 

However, if we want to detect several objects in the image, the function `cv.minMaxLoc()` won't give you all the locations. 

**QUESTION**  (/2)

Implement your own function, `multiMinMax()`, which takes an input 2D image `src` and returns an `output_array` with the locations of the minimums or maximums depending on the provided `flag` (`flag = "min"` or `flag = "max"`), and a given `params`. The `params` variable determines the number of matching objects to return. It can be, for example, a _threshold_ for the local minima/maxima, the maximum number of maximums/minimums to return, a difference between the global maxima/minima to be included, etc.

In [ ]:
def multiMinMax(src, flag, params):
    """
    Return the extrema of an image
    :src:       Image
    :flag:      Minima or maxima
    :param:  Parameters of your function
    return: The minimum and maximum
    """
    ret = None 
    
    #######################

    if flag not in ["min", "max"]:
        return "Invalid flag. Please choose 'min' or 'max'."

    global_min, global_max, _, _ = cv.minMaxLoc(src)
    # print(global_min, global_max)
    if flag == "min":
        diff_img = abs(global_min - src)/abs(global_max-global_min)
    else:
        diff_img = abs(src - global_max)/abs(global_max-global_min)
    # print(diff_img.shape)
    ret = np.where(diff_img <= params)
    return ret
    #######################


The next cell tests your implementation and draws the results.


In [ ]:
def drawROIS(src, template, locations, color = (0,0,255)):
    w, h = template.shape[::-1]
    # Make copy of image to draw on it without changing the original image
    canvas = src.copy();
    
    # Draw all rectangles
    for pt in list(zip(*locations[::-1])):
        # print(pt)
        cv.rectangle(canvas, pt, (pt[0] + w, pt[1] + h), color, 2)
       
    return canvas

# How should be called:
img_rgb  = cv.imread('../data/space-invaders_1.jpg')
img_gray = cv.imread('../data/space-invaders_1.jpg',0)
template = cv.imread('../data/template_0.png',0)

# For a given distance Map
distance_map_norm_corr = cv.matchTemplate(img_gray,template,cv.TM_CCOEFF_NORMED)
# Use your function here!
locations = multiMinMax(distance_map_norm_corr,'max',6e-2)
# print(locations)
# Draw the ROIs 
img_rgb_holder = drawROIS(img_rgb, template, locations)
# example:
display_image(img_rgb_holder)

In [ ]:
locations = multiMinMax(distance_map_norm_corr,'max',1e-2)
locations



Now that you are all set up, use the functions you implemented and *try* to detect ALL the matching objects in the input image (Using template Matching).

Rules:

* You can use any metric you want, self-implemented or from OpenCV. 
* You can tweak your `multiMinMax()` to get better results.
* For the points 1),2) and 3) below you have to do it in grayscale.
* 4) can use multi-channel heuristics.


#### 1) Perfect match <3

The next cell load the image and the templates. 


In [ ]:
# input images to use
p1_src_rgb    = cv.imread('../data/space-invaders_1.jpg')
p1_src_gray   = cv.imread('../data/space-invaders_1.jpg',0)

# Show the matching of these 2 templates:
p1_template_1 = cv.imread('../data/template_1.png',0)
p1_template_2 = cv.imread('../data/template_2.png',0)

# Image
display_image(p1_src_gray)
fig, ax = plt.subplots(1, 2, figsize=(3, 1),squeeze=False)

# Templates
display_image(p1_template_1, axes=ax[0][0])
ax[0][0].set_title("Template 1")
ax[0][0].set_xticks([])
ax[0][0].set_yticks([])

display_image(p1_template_2, axes=ax[0][1])
ax[0][1].set_title("Template 2")
ax[0][1].set_xticks([])
ax[0][1].set_yticks([])

**QUESTION** (/4)

Using the graycale template `p1_template_1` and `p1_template_2` provided in the previous cell, detect all the objects in the image `p1_src_gray` and show the locations of the matching objects in the image `p1_src_rgb`.

Follow the code below and provide some insights like:

* Why did you choose that given metric?  
* How robust to false positives/negatives is your selected metric.
* Is the number of output locations the same as the matching objects? 



In [ ]:
## YOUR CODE HERE
# Perform template matching for both templates
dist_map_template_1 = cv.matchTemplate(p1_src_gray, p1_template_1, cv.TM_CCOEFF_NORMED)
dist_map_template_2 = cv.matchTemplate(p1_src_gray, p1_template_2, cv.TM_CCOEFF_NORMED)

# Use multiMinMax function to get the locations of matching objects
locations_template_1 = multiMinMax(dist_map_template_1, 'max', 6e-2)
locations_template_2 = multiMinMax(dist_map_template_2, 'max', 6e-2)

# Draw the ROIs on the RGB image for both templates
img_rgb_with_ROIs_template_1 = drawROIS(p1_src_rgb, p1_template_1, locations_template_1)
img_rgb_with_ROIs_template_2 = drawROIS(img_rgb_with_ROIs_template_1, p1_template_2, locations_template_2)

# Display the image with detected objects
display_image(img_rgb_with_ROIs_template_2)
# print(len(locations_template_1[0]), len(locations_template_2))

**YOUR ANSWER**

#### 2) Not so perfect Match </3


The next cell load the image and the templates. 


In [ ]:
# input
p2_src_rgb    = cv.imread('../data/space-invaders_2.jpg')
p2_src_gray   = cv.imread('../data/space-invaders_2.jpg',0)

# 2 tempaltes (check that tempalte one don't match all the invaders in the same row) 
p2_template_1 = cv.imread('../data/template_3.png',0)
p2_template_2 = cv.imread('../data/template_1.png',0)

# Image
display_image(p2_src_gray)

# Templates
fig, ax = plt.subplots(1, 2, figsize=(3, 1),squeeze=False)
display_image(p2_template_1, axes=ax[0][0])
ax[0][0].set_title("Template 1")
ax[0][0].set_xticks([])
ax[0][0].set_yticks([])

display_image(p2_template_2, axes=ax[0][1])
ax[0][1].set_title("Template 2")
ax[0][1].set_xticks([])
ax[0][1].set_yticks([])

**QUESTION** (/4)

Using the graycale template `p2_template_1` and `p2_template_2` provided in the previous cell, detect all the objects in the image `p2_src_gray` and show the locations of the matching objects in the image `p2_src_rgb`. All the _invaders_ in the same row counts as the "same" class. 

Follow the code bellow and provide some insights like:

* How did you select the number of maximums/minimus?  
* How robust to false positives/negatives is your selected metric.
* Is the number of output locations the same as the matching objects? 
* Could you use any of the features from the last chapter to improve the matching?

In [ ]:
## YOUR CODE HERE
# Perform template matching for both templates
dist_map_template_1 = cv.matchTemplate(p2_src_gray, p2_template_1, cv.TM_CCOEFF_NORMED)
dist_map_template_2 = cv.matchTemplate(p2_src_gray, p2_template_2, cv.TM_CCOEFF_NORMED)

# Use multiMinMax function to get the locations of matching objects
locations_template_1 = multiMinMax(dist_map_template_1, 'max', 2.5e-1)
locations_template_2 = multiMinMax(dist_map_template_2, 'max', 3e-1)

# Draw the ROIs on the RGB image for both templates
img_rgb_with_ROIs_template_1 = drawROIS(p2_src_rgb, p2_template_1, locations_template_1)
img_rgb_with_ROIs_template_2 = drawROIS(img_rgb_with_ROIs_template_1, p2_template_2, locations_template_2)

# Display the image with detected objects
display_image(img_rgb_with_ROIs_template_2)

**YOUR ANSWER**

#### 3) Hidden objects game


Finding hidden objects in cluttered illustration is a popular casual game which develop your observation skills. In this exercise, you will use your template matching skills to develop an efficient object detector for this game.


We will play on scenes from the **Big Home Hidden Objects** game, in which image templates of the objects to find are shown at the bottom of the scene. We provide you the method `extract_big_home_templates()` to extract the grayscale template of each object given a scene.

The next cell load the image and the templates. 

In [ ]:
# DO NOT MODIFY THIS FUNCTION
def extract_big_home_templates(image):
    
    # We extract list of objects in a heuristic manner
    list_image = image[370:418, 77:712].copy()
    
    # There are 10 objects per image
    n_objects = 10
    stride = list_image.shape[1]/10
    
    # Extract each template image
    objects = []
    for i in range(n_objects):
        object_rgb = list_image[:,int(stride*i):int(stride*(i+1))][15:-15,15:-15]
        center_coordinates = (int(object_rgb.shape[1]/2), int(object_rgb.shape[0]/2))
        object_rgb = cv2.ellipse(object_rgb, center_coordinates, (40, 32), 0, 0, 360, (35, 35,35), 20)
        object_gray = cv2.cvtColor(object_rgb, cv2.COLOR_BGR2GRAY)
        objects.append(object_gray)
        
    return objects

In [ ]:
# inputs
hidden_rgb  = cv.imread('../data/hidden.jpeg')

# get list of templates
templates = extract_big_home_templates(hidden_rgb)
hidden_rgb = hidden_rgb[80:360]
hidden_gray = cv2.cvtColor(hidden_rgb, cv2.COLOR_BGR2GRAY)

fig, ax = plt.subplots(1,1, figsize=(10, 10),squeeze=False)

display_image(hidden_rgb, axes=ax[0][0])
ax[0][0].set_title("Input")
ax[0][0].set_xticks([])
ax[0][0].set_yticks([])

plt.show()

fig, ax = plt.subplots(1,10, figsize=(20, 20),squeeze=False)

for i, template in enumerate(templates):
    display_image(template, axes=ax[0][i])
    ax[0][i].set_title("Templates {:d}".format(i+1))
    ax[0][i].set_xticks([])
    ax[0][i].set_yticks([])


**QUESTION** (/10)

Your task is to use these templates and what you have learned so far to find the hidden objects in the given illustration `hidden_gray`.

Plot the image `hidden_rgb` showing the location of each object and report the number of miss. Note each object is present once in the scene. So you can write a variant of `multiMinMax()` (e.g `minMax()`) which returns the global minimum or maximum of a 2D `src` image according to the provided `flag` (`flag = "min"` or `flag = "max"`). You can reuse the function `cv.minMaxLoc()`.


In this exercise, you may choose to use any transformation in the input image or tweak the distance metric. You can't modify the templates (only scale it).

Follow the code below and provide some insights like:

* What metric seemed to work better this time ? 
* Was it different from the previous exercise ?
* How many objects did you find ? Why these ones in partiular ?

Save the output in a separate image for easier visualization.

In [ ]:
def minMax(src, flag):
    
    #YOUR CODE HERE
    if flag not in ["min", "max"]:
        return "Invalid flag. Please choose 'min' or 'max'."
    
    global_min, global_max, min_loc, max_loc = cv.minMaxLoc(src)
    
    if flag == "min":
        return (global_min, [[min_loc[1]], [min_loc[0]]])
    else:
        return (global_max, [[max_loc[1]], [max_loc[0]]])

In [ ]:
#YOUR CODE HERE


def find_objects(hidden_gray, hidden_rgb, templates):
    found_objects = []
    for i, template in enumerate(templates):
        dist_map = cv.matchTemplate(hidden_gray, template, cv.TM_CCOEFF_NORMED)
        val, loc = minMax(dist_map, 'max')
        # max_loc = ([max_loc[1]], [max_loc[0]])
        

        found_objects.append((i, loc))
        img_with_ROI = drawROIS(hidden_rgb, template, loc)
        print("Found template", i+1, "at", loc[::-1], "with value", val)
        hidden_rgb = img_with_ROI.copy()
    
    return hidden_rgb, found_objects


# Find the objects
result_img, found_objects = find_objects(hidden_gray, hidden_rgb, templates)

# Display the image with detected objects
fig, ax = plt.subplots(1,1, figsize=(20, 20),squeeze=False)

display_image(result_img, axes=ax[0][0])
ax[0][0].set_title("Input")
ax[0][0].set_xticks([])
ax[0][0].set_yticks([])

plt.show()
fig, ax = plt.subplots(1,10, figsize=(20, 20),squeeze=False)

for i, template in enumerate(templates):
    display_image(template, axes=ax[0][i])
    ax[0][i].set_title("Templates {:d}".format(i+1))
    ax[0][i].set_xticks([])
    ax[0][i].set_yticks([])

**YOUR ANSWER**

#### 4) Find all hidden objects

As you can see, using template matching can be tricky, even when you have _good_ templates. As for the last exercise, you will try to design good __templates__  in order to find all hidden objects in the image. In this exercise, you can (if you want) use the information of the 3 RGB channels (to generate a 1 _better_ 1 channel image for example) and transform the input image (for example to homogenize the scale!). You can reuse any features from the last chapter to improve the matching.

**QUESTION** (/6)

Plot your selected templates and the input image _showing_ the location of each object (if you can) and any miss if any.

Follow the code below and provide some insights like:

* Was it different from the previous exercise?
* What did you improve ?

In [ ]:
# inputs
hidden_rgb  = cv.imread('../data/hidden.jpeg')

# get list of templates
hidden_rgb = hidden_rgb[80:360]
hidden_gray = cv2.cvtColor(hidden_rgb, cv2.COLOR_BGR2GRAY)

fig, ax = plt.subplots(1,1, figsize=(10, 10),squeeze=False)

display_image(hidden_rgb, axes=ax[0][0])
ax[0][0].set_title("Input")
ax[0][0].set_xticks([])
ax[0][0].set_yticks([])

plt.show()

fig, ax = plt.subplots(1,10, figsize=(20, 20),squeeze=False)

#######################

# YOUR CODE HERE TO LOAD YOUR TEMPLATES
# def preprocess_image(image):
#     # gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
#     blurred = cv.GaussianBlur(image, (3, 3), 0)
#     edges = cv.Canny(blurred, 5, 40)
#     return edges
# edges_templates = [preprocess_image(t) for t in templates]
templates = 
for i, template in enumerate(templates):
    display_image(template, axes=ax[0][i])
    ax[0][i].set_title("Templates {:d}".format(i+1))
    ax[0][i].set_xticks([])
    ax[0][i].set_yticks([])
#######################


In [ ]:
## YOUR CODE HERE
\
def preprocess_image(image):
    # gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    blurred = cv.GaussianBlur(image, (5, 5), 0)
    edges = cv.Canny(blurred, 50, 150)
    return edges

hidden_edges = preprocess_image(hidden_gray)

scaled_templates = [cv.resize(t, (int(t.shape[1] * 1.1), int(t.shape[0] * 1.1))) for t in templates]

result_img, found_objects = find_objects(hidden_gray, hidden_rgb, scaled_templates)

# Display the image with detected objects
fig, ax = plt.subplots(1,1, figsize=(20, 20),squeeze=False)

display_image(result_img, axes=ax[0][0])
ax[0][0].set_title("Input")
ax[0][0].set_xticks([])
ax[0][0].set_yticks([])

plt.show()
fig, ax = plt.subplots(1,10, figsize=(20, 20),squeeze=False)

for i, template in enumerate(templates):
    display_image(template, axes=ax[0][i])
    ax[0][i].set_title("Templates {:d}".format(i+1))
    ax[0][i].set_xticks([])
    ax[0][i].set_yticks([])

**YOUR ANSWER**

## 2.2 Person Detection
(*25 points*)

In this section, we will return to the HOG features from the last Chapter. As we said before, HOG was proposed as a useful feature for human detection. If you reach this point, you may have noticed that Template matching may not be the best option for this. Imagine how difficult it would be to create a template for any human-shaped structure that you would like to detect as a human in a scene. Instead of that, you will train a Linear Classifier from scratch.

In this section, you will use OpenCV's implementation to extract the HOG's features of the INRIA's Persons dataset to train an SVM Linear classifier (https://en.wikipedia.org/wiki/Support_vector_machine). For this, we will train the SVM classifier using the Scikit-learn Machine Learning library (http://scikit-learn.org).

### 2.2.1 Dataset 

 This dataset was collected as part of the research work on detection of upright people in images and video. The research is described in detail in the CVPR 2005 paper _Histograms of Oriented Gradients for Human Detection_. The full dataset is about ~1 GB and contains several thousands of pedestrian images. 
 
For your convenience, the dataset is already separated into two sets: 
* "**_Positives_**" which are all the images containing at least one person. 
* "**_Negatives_**" any kind of non-human shaped objects images.

In addition, the data is already separated in a **training** and **testing** set.


You can download the dataset on Moodle and put the `.tar` file into the `data` folder. To uncompress the data, you can either use the function below or directly do it in your file explorer, it is up to you.

In [ ]:
def maybe_extract(filename, force=False):
    """
    Uncompress a given *.tar file
    :param filename: File to be uncompressed
    """
    # remove .tar.gz
    root = os.path.splitext(os.path.splitext(filename)[0])[0] 
    if os.path.isdir(root) and not force:
        # You may override by setting force=True.
        print('%s dataset (seems to be) already present.\nSkipping extraction of %s.' % (root, filename))
    else:
        print('Extracting data for %s. This may take a while. Please wait.' % root)
        tar = tarfile.open(filename)
        tar.extractall(os.path.dirname(filename))
        tar.close()
    print("All setup.")

In [ ]:
# Optional
maybe_extract(filename='../data/INRIAPerson.tar')

### 2.2.2 Features Extraction

Once you have the data, you will now process each image using the OpenCV HOGDescriptor implementation [[doc](https://docs.opencv.org/4.0.0/d5/d33/structcv_1_1HOGDescriptor.html)]. 

In the next cell, we define the parameters of the HoG descriptor and load the path to the images with the function `return_img_path()`.




In [ ]:
def return_img_path(sample_file, folder):
    with open(sample_file, 'rt') as f:
        img_path = []
        for im_path in f:
            if im_path.strip():
                path = os.path.join(folder, im_path.strip())
                img_path.append(path)
    return img_path
    

# Data location
#Positive folder:
pos_im_path = "../data/inria_train_pos.txt"
#Negative folder:
neg_im_path = "../data/inria_train_neg.txt"
    

path_img_pos = return_img_path(pos_im_path, folder='../data/INRIAPerson')   
path_img_neg = return_img_path(neg_im_path, folder='../data/INRIAPerson')   


# Image descriptor parameters
# ---------------------------
# Window size: this specifies the size of the input image (remember to scale the input to this size!)
win_size     = (64,128)
# Size on pixels of each block (remember that a block contains a set of CELLS)
block_size   = (16,16)
# The separation between each block. If this value is less than the block size, 
# there will be overlapping blocks. 
block_stride = (8,8)
# The size of each CELL. Each cell computes one histogram. The cells should FIT inside a block.
cell_size = (4,4)
# Number of bins for each histogram.
nbins = 10


**QUESTIONS (/3)** 

In the next cell, implement the function `_extract_feature()` that takes as input a list of path to images, the HoG descriptor, the label of the images and the input size of the HoG descriptor. The function returns a list of HoG features for each image and a list of corresponding label. 

**WARNING**: The dataset may contain corrupted images. Be sure, inside your code, to check if the image was loaded properly. Otherwise, you will get either trash features or execution errors.


Sometimes OpenCV can not load the image properly, that's why we're gonna use another package to load them. The command below shows how to achieve the same as `cv2.imread(...)` function.

```python
np.flip(np.asarray(imageio.imread('<image_path>', dtype=np.uint8))[..., :3], -1)
```

The `imread(...)` function load the image, it is then converted to numpy array with `np.asarray()`. Only the first 3 channels are selected with `[..., :3]` and finally it is converted from `RGB` to `BGR` with the `np.flip(..., -1)` function. 

In [ ]:
def _extract_feature(img_path, descriptor, label, hog_size):
    
    features = []
    labels = []
    
    ## YOUR CODE HERE
    
    ### Test if image loaded properly
    for p in img_path:
        image = np.flip(np.asarray(imageio.imread(p)).astype(np.uint8)[..., :3], -1)
        resized = cv.resize(image, hog_size, interpolation = cv2.INTER_AREA)
        ### Transfom to grayscale
        image = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
        HOGs = descriptor.compute(image)
        features.append(HOGs)
        labels.append(label) 
    
    return features, labels

The next cell loads the HoG features and labels for the positive and negatives images of the training set.

**QUESTIONS (/2)**
- The length of each individual feature vector should be of 16800. Why?
- What should be the size of the _training_features_ list? Why?


In [ ]:

# hog is an instance taht contains the info and is able to compute the feature vector.
hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)


# lists to save the features and labels.
training_features = []
labels = []

# Positive features
print ("Calculating the descriptors for the positive samples and saving them")
pos_feat, pos_label = _extract_feature(path_img_pos, descriptor=hog, label=1, hog_size=win_size)   
training_features.extend(pos_feat)
labels.extend(pos_label)

#Negative features
print ("Calculating the descriptors for the negative samples and saving them")
neg_feat, neg_label = _extract_feature(path_img_neg, descriptor=hog, label=0, hog_size=win_size)   
training_features.extend(neg_feat)
labels.extend(neg_label)

# Summary
print('#Samples: {}'.format(len(training_features)))
print('Feature size: {}'.format(training_features[0].shape))


**YOUR ANSWER**
- The length of each individual feature vector should be of 16800. Why?

This is because of the size of the image, of the cells, the number of bins of the histograms and the block stride. We can compute it as follow:
- Image size = (64, 128)
- Block size = (16, 16)
- Block stride = (8, 8)

This means that the block can be slided **15** times on the horizontal axis (sliding a 16x16 block with steps of size 8 can be done 15 times on width of 128). The block can be slided **7** times on the vertical axis (sliding a 16x16 block with steps of size 8 can be done 7 times on height of 64). Each time we are sliding a block, we compute **16** HoG (one per cell in the block) whose size is **10**.

Multiplying out the whole leads to the final dimension of the feature vector of the image: `dim` = `15` * `7` * `16` * `10` = `16800`

- What should be the size of the _training_features_ list? Why?

This list should have a length of 3634, since we have grouped all images together. There are 2416 positive images listed in the file `inria_train_pos.txt` and 1218 negative images listed in the file `inria_train_neg.txt`, so `2416`+`1218`=`3634`. Each of these sample has 16'800 dimensions as it is the feature vector of an image. 

### 2.2.3 Classifier

Now that you have the HoG features, you'll train a Linear Support Vector Classifier [[doc](
http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html)] using Scikit-learn. 

Please take the time to read the implementation details, but more importantly, the examples and theory provided in the documentation. 

**QUESTION (/5)** 

In the next cell, train a SVM classifier on the HoG features.

In [ ]:
# Stack features
train_features = np.asarray(training_features)
train_labels = np.asarray(labels)

# Train classifier
# Check the dimentions first
print('Feature matris dims: {}, labels dims: {}'.format(train_features.shape, 
                                                        train_labels.shape))

#########################
## YOUR CODE HERE
# create a LINEAR classifier instance here (LinearSVC): 

model = LinearSVC(loss='hinge',max_iter=10000,C=0.01)
# Train the classifier (LinearSVC.fit).
print ("Training a Linear SVM Classifier")
model.fit(training_features, labels)

print('Done')


#########################

#### Justification of the parameters

In this case we chose a `C = 1.0` which is the default parameter for LinearSVC for sklearn and we extended max_iter to `10'000` (the default value is `1'000`). This extension was made for the sake of the accuracy. Enabling more iterations leads to a longer computation time but also increases the precision.

### 2.2.4 Evaluation

After training the classifier, you can use this Support Vector Machine to *classify* if an * HOG feature vector* (with strictly the same dimensions as your training data) comes from an image with a human-shaped form in it (prediction = 1), or not (prediction = 1).


**QUESTIONS (/10)**
In the next cell, for each image in the test folders: 
* Compute the HOG feature vector,
* Predict/classify the vector as positive (1) or negative (0),
* Compute the estimation error for the negative and positive images _separetely_,
* Compute and report the F1-score [[doc](https://en.wikipedia.org/wiki/F1_score)]

In [ ]:
test_neg_path = "../data/inria_test_neg.txt"
test_pos_path = "../data/inria_test_pos.txt"

###################
# YOUR CODE HERE 
print ("Estimating the test data [Negative samples]")
# Extract features
# Predict labels 
# Prediction Error 

from sklearn.metrics import accuracy_score

path_img_neg = return_img_path(test_neg_path, folder='../data/INRIAPerson')   
test_neg_feat, label_neg_test = _extract_feature(path_img_neg, hog, label=0, hog_size=win_size)

test_features_n = np.asarray(test_neg_feat)
test_labels_n = np.asarray(label_neg_test)

prediction_neg = model.predict(test_features_n)
tn = accuracy_score(prediction_neg, test_labels_n)          
print("Success rate negative samples:", tn)

print ("Estimating the test data [Positive samples]")
# Extract features
# Predict labels 
# Prediction Error
###################
                               
path_img_pos = return_img_path(test_pos_path, folder='../data/INRIAPerson')
test_pos_feat, label_pos_test = _extract_feature(path_img_pos, hog, label=1, hog_size=win_size)
                               
test_features_p = np.asarray(test_pos_feat)
test_labels_p = np.asarray(label_pos_test)

prediction_pos = model.predict(test_features_p)
tp = accuracy_score(prediction_pos, test_labels_p) 
print("Success rate positive samples:", tp)

### F1 score
fn = 1-tn
fp = 1-tp
err = 2*tp/(2*tp+fn+fp)

# Compute score
print('Estimation Error: {:.3f}%'.format(err * 100.0))
print('Success Rate: {:.3f}%'.format(err * 100.0))

The basic classifier above can (should) achieve a success rate of 89% for the positive and less than 2% error for the negative images respectively. 

**QUESTION** (/5)
Can you tweak the HOG parameters to improve a little bit?

First we will try to implement the same method using a higher number of bins. The performance should increase since adding bins leads to more accurate prediction as the histograms will detect changes along more directions. Each bin of the histogram corresponds to the variation of the gradient amplitude along a certain range of the phase. Increasing the number of bins implies that those variations of the gradient amplitude in the image will be noticed in a higher number of directions since the phase range will be smaller which increases the accuracy. Thus, the detection performance will increase. 

In [ ]:
## YOUR CODE HERE

### Tune HOG values
win_size     = (64,128)
block_size   = (16,16)
block_stride = (8,8)
cell_size = (4,4)
nbins = 16  ### Take larger number of bins
new_hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)

### Train feature extraction
training_features = []
labels = []
print ("Calculating the descriptors for the positive samples and saving them")
pos_feat, pos_label = _extract_feature(path_img_pos, descriptor=new_hog, label=1, hog_size=win_size)   
training_features.extend(pos_feat)
labels.extend(pos_label)

#Negative features
print ("Calculating the descriptors for the negative samples and saving them")
neg_feat, neg_label = _extract_feature(path_img_neg, descriptor=new_hog, label=0, hog_size=win_size)   
training_features.extend(neg_feat)
labels.extend(neg_label)

new_model = LinearSVC(loss='hinge',max_iter=10000,C=0.01)
print ("Training a Linear SVM Classifier")
new_model.fit(training_features, labels)

### Test features extraction

path_img_neg = return_img_path(test_neg_path, folder='../data/INRIAPerson')   
test_neg_feat, label_neg_test = _extract_feature(path_img_neg, new_hog, label=0, hog_size=win_size)

test_features_n = np.asarray(test_neg_feat)
test_labels_n = np.asarray(label_neg_test)

print ("Estimating the test data [Negative samples]")
prediction_neg = new_model.predict(test_features_n)
tn = accuracy_score(prediction_neg, test_labels_n)          
print("Success rate negative samples:", tn)

print ("Estimating the test data [Positive samples]")                               
test_pos_feat, label_pos_test = _extract_feature(path_img_pos, new_hog, label=1, hog_size=win_size)
                               
test_features_p = np.asarray(test_pos_feat)
test_labels_p = np.asarray(label_pos_test)

prediction_pos = new_model.predict(test_features_p)
tp = accuracy_score(prediction_pos, test_labels_p) 
print("Success rate positive samples:", tp)

### F1 score
fn = 1-tn
fp = 1-tp
err = 2*tp/(2*tp+fn+fp)

# Compute score
print('Estimation Error: {:.3f}%'.format(err * 100.0))
print('Success Rate: {:.3f}%'.format(err * 100.0))

### Try to change the phase : 360° -> 180°
### cf toward datascience
### preprocess the data
### previous notebook

We now observe that the performance is much better using a larger number of bins.

Finally, from the description above, what we created is no more than a *classifier* for only 2 classes (binary classifier): human(human-shaped) or not. In order to create a functional Person detector for arbitrary images or video sequences, some engineering techniques (heuristics) need to be implemented. 

Pretty much as in template matching, in order to find a person in an arbitrary image you will need to: 

* Slide your classifier over the full area of the image.
* Detect possible matchings. 
* Report them as positive or negatives
* And optionally, repeat the procedure above in different scales, to assure multiscale detection!.

The procedure is nicely depicted in the image below for face detection.

<img src="https://www.pyimagesearch.com/wp-content/uploads/2015/03/sliding-window-animated-adrian.gif"> 
</img>
_Image taken from: https://www.pyimagesearch.com_

For now, we will leave those implementation details for later.

## 2.3 Face Recognition
(*35 points*)

In this section you will implement a face regognition algorithm using two approaches: `Eigen Faces` and `Fisher Faces` . These methods are both based dimensional reduction technics listed below:

- Principal Component Analysis ([*PCA*](http://www.utdallas.edu/~herve/abdi-awPCA2010.pdf))
- Linear Discriminant Analysis (*LDA*)

In general, in order to train a recognizer, several steps are needed and can be grouped as follow:

- Data preparation
- Recognizer training
- System validation

During validation, the system is tested with **unseen** data, i.e. the data hadn't used during the training phase, which ensures a fair performance assessment without biais. However this does not garantie that the system will *generalize* well to other dataset.

### 2.3. 1 Data Preparation

Data preparation covers various aspect of pre-processing step for training a system. At first, the images need to be splitted into two disctinct subsets thant will be used for `training` and `testing`. In our experiment the dataset used is the *Yale dataset version B*, it includes a total of 38 different identity (*i.e. subject*) each having 20 images undergoing different illumination condition for a total of 760 samples. We split the data into `training` and `testing` sets by selecting randomly samples from each subjects and placed into the corresponding subset. Special care needs to be taken in order to avoid having the same example multiple time.

The first task is to gather the labels and the images that will be used to train the system. One solution is to store these information into a dictionary where the identity is the key and the pathes to the images for this subject are the values. 

**QUESTION** (/2)

In the next cell, implement the function `load_dataset()` that takes as input the path to the dataset, and returns a dictionary in which the keys are the person ID and the values are the pathes to the corresponding images.

In [ ]:
def load_dataset(path):
    """
    Scan for images in a given `path` and extract the label as well

    :param path:    Path where YaleB dataset is stored
    :return:        Dictionary storing the ID and a list of images for this ID
    """
    data = {}
    
    # YOUR CODE HERE
    for i in range(1, 40):
        if i < 10:
            path = os.path.join('..', 'data', 'yaleB', 'yaleB0'+str(i))
            data[i] = path
        elif i==14: ### 14 is not present in the folders
            pass
        else:
            path = os.path.join('..', 'data', 'yaleB', 'yaleB'+str(i))
            data[i] = path
        
    return data

The next cell load the data.

In [ ]:
# Load data
data = load_dataset(os.path.join('..', 'data', 'yaleB'))
assert(len(data) == 38)

When all the images and labels have been gathered, the next step is to split into two subsets, the train set and test set. The training set will be composed of $75\%$ of the images of each subject and the remaining $25\%$ will be used as test set.

Once again the two subsets information will be stored into two separates disctionary similar to what has been done earlier.


**QUESTION** (/3)

In the next cell, implement to the function `split_dataset()` that takes as input the dictionary of data and the ratio between training and testing sets, and returns two dictionaries, one with the training data and one with the testing data.

In [ ]:
def split_dataset(data, ratio):
    """
    Splipt randomly a dataset into two subset. The ratio provide the distribution for each subset

    :param data:    Overall dataset
    :param ratio:   Split ratio
    :return:        Two dictionaries, train/test
    """
    train = {}
    test = {}
    
    # YOUR CODE HERE
    import random
    
    for key in data.keys():
        
        ### Remove the Icon_ file from the pictures
        files = [file for file in os.listdir(data[key]) if file.endswith(".pgm")]
        
        ### Shuffle the pictures before splitting
        random.shuffle(files)
        
        for i, pict in enumerate(files):
            if i == 0:
                train[key] = []
                train[key].append(os.path.join(data[key], pict))
            elif i < int(ratio*len(data[key])):
                train[key].append(os.path.join(data[key], pict))
            elif key in test.keys():
                test[key].append(os.path.join(data[key], pict))
            else:
                test[key] = []
                test[key].append(os.path.join(data[key], pict))
        
                
    return train, test

The next cell splits the data.

In [ ]:
train, test = split_dataset(data, 0.75)
assert(len(train) == 38)
assert(len(test) == 38)

Now with these two subsets we can load the images and extract features from them. In this case the pixel intensity of grayscale images will be used as feature, therefore for an image $I \in \mathbb{R}^{ w \times h}$ the feature vector will have a size of $wh$. This value can be quite large very easily, therefore all images will be downsampled by a factor of $2$.

All the training samples will be concatenated into a single matrix where each row is an image (*i.e. flattened*) with dimensions $N \times K$ where $N$ is the number of samples and $K$ is the dimension of a single image, $K = \frac{wh}{4}$. Also, make sure the features are float32 normalized between 0 and 1. 

The corresponding labels will also be concatenated into a single vector of dimension $N \times 1$.


**QUESTION** (/5)

In the next cell, implementto the function `load_images()` that takes as input a dictionary of data, and returns the feature matrix and the label vector. 

In [ ]:
from sklearn import preprocessing as pre

def load_images(subset):
    """
    Load images into one single matrix where each row is one single image (flattened). The final dimensions is [N x K]
    where N is the numper of samples available and K is the number of pixel in one image. The original image is first
    downspample by a factor of 2

    Labels are also exported into a single vector of dimensions [N x 1]

    :param subset: Dictionary storing labels/images
    :return:       Data matrix and label vector
    """
    
    data = None
    label = None
    
    # YOUR CODE HERE
    label = []
    data = []
    for i in range(1, len(subset)+2):
        
        ### No folder yaleB14
        if i != 14:
            for file in subset[i]: 
                
                ### Ensure take a .pgm file
                if file.endswith(".pgm"):
                    image = cv2.cvtColor(cv.imread(file), cv2.COLOR_BGR2GRAY).astype(np.float32)
                    dim = image.shape
                    new_im = cv2.resize(image, (int(dim[1]/2), int(dim[0]/2)))
                    x = new_im.flatten()
                    x_norm = (x-np.min(x))/(np.max(x)-np.min(x))
                    data.append(x)
                    label.append(i)
                    
    data, label = np.array(data), np.array(label)    

    return data, label

The next cell loads the training data and the labels. 

In [ ]:
# Load training images into memery
train_img, train_label = load_images(train)
# Sanity check
assert(train_img.shape[0] == train_label.size)
# Output number of samples
print("There is a total of {} samples for the training set".format(train_label.size))


### 2.3.2 Eigenfaces

To perform recognition, all pixel's intensities are used as feature vector. The dimension of theses descriptors will be large, therefore a *clever* representation of the data, called subspace, is needed. 

This subspace is computed using *Principal Component Analysis* method in order to extract meaningfull information and reduce the dimension of the problem. The *PCA* approach is completely unsupervised and extract the directions, or *basis*, where the variation in the data are the largest inside the feature space. 

Since we are interested in the variation in the data, the first step is to remove the commmon information present in all samples by subtracting the **average face**. The average is computed using all training samples $I_i$ as follow:

$$
\bar{\boldsymbol{I}} = \frac{1}{N_t} \sum_{i=0}^{N_t} \boldsymbol{I}_i
$$

where $N_t$ is the total number of training samples and $I_i$ is a specific training sample. Then each samples $I_i$ are normalized as follow:

$$
\boldsymbol{\phi_i} = \boldsymbol{I}_i - \bar{\boldsymbol{I}}
$$

With all samples normalized, we need to find a set of orthognonal basis which best explain the distribution of our data. To do so we compute the eigendecomposition of the covariance matrix of the normalized samples.

$$
\boldsymbol{C} = \frac{1}{N_t - 1} \sum_{i}^{N_t} \boldsymbol{\phi}_i\boldsymbol{\phi}_i^{\top} = \frac{1}{N_t - 1} \boldsymbol{\Phi\Phi}^{\top}, \quad \text{where } \Phi = \left[\boldsymbol{\phi}_0, \dots, \boldsymbol{\phi}_{N} \right]
$$

Find the eigenvectors $u_k$ and the eigenvalues $\lambda_k$. 

So far the dimensions of the problem have not been reduced. Moreover the size of the covariance matrix will be $K \times K$ with $K = \frac{wh}{4}$. Therefore we will find $K$ eigenvectors representing the variation in our data. To reduce the dimension we will select only the eigenvectors that contribute the most to the variation and dropping the one with little influence. Doing so will reduce the dimension of the subspace to $K \times K_m$.

The question is how to properly determine this $K_m$ value. It can be done by using the eigenvalues computed earlier. These values are representing the energy each vector contribute for. Therefore it is possible to dertmine the number of basis to select in order to retain a certain amount of energy.

$$
\frac{\sum_{k=0}^{K_m}\lambda_k}{\sum_{i=0}^{K}\lambda_i} < \Theta
$$

Where $\Theta$ represents the amount of energy to retain, which usually is around $95\%$ but can vary depending on the application. 
Finally the subspace is defined as:

$$
\boldsymbol{U} = \left[\boldsymbol{u}_0, \dots, \boldsymbol{u}_{K_m}\right], \quad \boldsymbol{U} \in \mathbb{R}^{K \times K_m}
$$

In practice, the PCA decomposition is computed using `sklearn.decomposition.PCA`, more information can be found [here](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html).


**QUESTION** (/3)

In the next cell, compute the face subspace on the training data by retaining $95\%$ of the variance present in the training data.

Once the subspace is computed, display the first $8$ modes or *eigenfaces*. 
Comment on what you see. What do you thing are the limitations of such approach ?


In [ ]:
# YOUR CODE HERE
train_centered = train_img-train_img.mean(axis=0)
#plot_gallery("Faces from dataset", train_centered[:6])
pca = PCA()
pca.fit(train_centered)
#pca_estimator = PCA(n_components=8, svd_solver="randomized", whiten=True)
#pca_estimator.fit(train_centered)

m, summ = 0, 0

for i, var in enumerate(pca.explained_variance_ratio_):
    summ += var
    if summ>0.95:
        m = i
        break
        
print("The total number of components to retain 95% of the variance of the data is:", m)
eigenfaces = pca.components_[:m]    
print(eigenfaces.shape)
dim = cv.imread(train[1][0]).shape[:2]


for i in range(8):
    plt.subplot(2,4,i+1)
    plt.title("EigenFace #"+str(i))
    plt.imshow(eigenfaces[i, :].reshape(int(dim[0]/2), int(dim[1]/2)), cmap='gray') ### Eigenfaces instead of pc I think
    plt.axis('off')
plt.show()

In [ ]:
### PCA from scratch 

X = train_img
mean = np.matmul(np.ones(X.shape[0]).reshape(X.shape[0],1),X.mean(0).reshape(1,X.shape[1]))
X = X - mean

S = np.matmul(X, X.transpose())*(1/X.shape[0])
w, v = np.linalg.eig(S)

total_var = np.cumsum(w)/w.sum()
num1 = total_var.shape[0] - (total_var > 0.95).sum()
num2 = total_var.shape[0] - (total_var > 0.99).sum()
red1 = np.round(100*(1-num1/total_var.shape[0]),2)
red2 = np.round(100*(1-num2/total_var.shape[0]),2)
print('To represent 95% of the total variation, we need ' + str(num1) + ' eigenvectors out of ' + str(total_var.shape[0]) + ' ( ' + str(red1) + '% reduction )' )
print('To represent 99% of the total variation, we need ' + str(num2) + ' eigenvectors out of ' + str(total_var.shape[0]) + ' ( ' + str(red2) + '% reduction )' )

#Get the principal components and plot
plt.plot(w/w.sum())
plt.title('Individual eigenvalue contribution')
plt.ylabel('Percent contribution to variance')
plt.xlabel('Eigenvalue number')
plt.show()

#Calculate the total percentage of variance explained
total_var = np.cumsum(w)/w.sum()
plt.figure()
plt.plot(total_var)
plt.title('Cumulative eigenvalue contribution')
plt.ylabel('Percent contribution to variance')
plt.xlabel('Eigenvalue number')
plt.show()

In [ ]:
u, s, v = np.linalg.svd(X)
pc = v.copy()
for i in range(8):
    plt.subplot(2,4,i+1)
    plt.title("EigenFace #"+str(i))
    plt.imshow(pc[i, :].reshape(int(dim[0]/2), int(dim[1]/2)), cmap='gray') ### Eigenfaces instead of pc I think
    plt.axis('off')
plt.show()

**YOUR ANSWER**

First, we notice that both methods using sklearn and computed from scratch leads to the same results. We need the `48` first components to retain `95% of the variance` present in the data. We can also read it on graph 2 that represents the cumulated contribution of the eigenvalues.

On the displayed images, we can distinguish 8 faces with a different illumination on each picture. For instance `EigenFace #0` is illuminated from the right while `EigenFace #1` is illuminated from the left and so forth. This variation in the illumination helps to distinguish the different features of the faces. For example, on `EigenFace #0` we can barely see the eyebrows on the face while they are easily observable on `EigenFace #3`.

In classification problem, it is always interesting to visualize the feature subspace to evaluate if they discriminate properly the different subjects. Having proper plots of an $N$ dimensional space is not feasable, and, therefore, we use only a few components (*i.e. 2 or 3*) of our projected samples. 


We want a subspace that is able to separate and cluster properly each subject to avoid miss recognition. The code snippet below shows the $3^{rd}$ and $4^{th}$ components on a $2D$ plane.

**QUESTION** (/2)

Do we have clean inter-subject separation ? Comment the results.


In [ ]:
# Poject data onto subspace
proj_train = pca.transform(train_img)

# Visualize 
# Colors for distinct individuals
colors = LabelEncoder().fit_transform(train_label.ravel())

plt.figure(figsize=(12, 7))
plt.scatter(proj_train[:, 2], proj_train[:, 3], c=colors)
plt.xlabel('PC2')
plt.ylabel('PC3')
plt.title('Trainset clusters')

**YOUR ANSWER**

On the plot `Trainset clusters`, we can observe that the different colour points are mixed and not clearly seperable. Even if they each colour tends to be concentrated within the same region, there are other colours that are grouped almost at the same place. That means that we cannot find a clean inter-subject seperation at that point.

In this new subspace, each training sample has a representation, given by its projection into the eigen subspace (*i.e. eigenface*) as follow:

$$
\boldsymbol{\omega}_i = \boldsymbol{U}^\top \boldsymbol{\phi}_i
$$

In the training set, there are multiple samples avaible for each subject. Their projection won't be exactly the same, and, therefore, we need a more generic representation of each person. One option is to average all representation of the specific person:

$$
\boldsymbol{\Omega}_k = \frac{1}{N_k} \sum_{p} \boldsymbol{\omega}_p
$$

where $N_k$ is the number of samples for subject $k$ and $\boldsymbol{\omega}_p$ represents the projected samples of subject $k$. In our case, $k$ goes from $0$ to $37$.

**QUESTION** (/3) 

In the next cell, implement the function `compute_centroids()` that takes as input the feature matrix, the transformation to be applied to the feature matrix and the corresponding label, and returns each subject's centroid, and the corresponding labels.




In [ ]:
def compute_centroids(data, trsfrm, label):
    """
    Given a list of training samples, compute the centroids of each uniques labels

    :param data:    Matrix with all feature vectors stored as row
    :param trsfrm:  Embeddings to use (object with `transform(X)` method available such as PCA/LDA from sklearn)
    :param label:   List of corresponding labels
    :return:        Centroids, unique labels
    """

    centroids = None
    unique_lbl = None

    # YOUR CODE HERE
    
    ### Try U = pca.component_[:m]
    data_centered = data-data.mean(axis=0)
    trsfrm.transform(data) # U = 
    U = trsfrm.components_[:m]
    centroids = np.array(np.split(U@data_centered.T, 38, axis=1)) # not .T and axis =0
    print(centroids.shape)
    centroids = np.mean(centroids, axis=2) # axis = 2
    
    unique_lbl = np.unique(label)
    print(centroids.shape)
    
    return centroids, unique_lbl

The next cell calculates the centroid of each subsject. 

**QUESTION** (/1)

What is the number of centroid? Why?

In [ ]:
# Define centroids
train_centroids, train_centroid_label = compute_centroids(train_img, pca, train_label)

In [ ]:
print("The number of centroids is :", train_centroids.shape[0])

We should have `38 centroids` since this is the number of subjects. As we computed the mean of all the projections of the training samples for each subject, this leads to 38 centroids and this is in agreement with the result printed above.


So far we've learned a face representation and computed the descriptors for the training samples. Ultimately, we want to use them to recognize face. The first step is to bring the *new* sample into our face *subspace*, similarly to the training samples:

$$
\boldsymbol{\omega}_n = \boldsymbol{U}^{\top}(\boldsymbol{I}_n - \bar{\boldsymbol{I}})
$$

where $U$ is the face subspace, $\bar{\boldsymbol{I}}$ is the average face learned on the training data, and $\boldsymbol{I}_n$ is the new sample to recognize.

In this subspace, we can measure the **similarity** (*distance*) between a testing sample and the centroids $\boldsymbol{\Omega}_k$ computed before. The predicted label will be the one corresponding the the closest centroid. 

$$ 
min \left|\left| \boldsymbol{\omega}_n - \boldsymbol{\Omega}_k \right|\right| \quad \forall k \in \{Train\}
$$


**QUESTION** (/2)

In the next cell, implement the function `recognize` that takes as input the transformation, the centroids, the centroid labels and the new samples, and returns the predicted labels of the new samples.


In [ ]:

def recognize(trsfrm, centroids, centroids_label, samples):
    """
    Perform object recognition on a given list of ``amples

    :param trsfrm:          Embeddings to use (object with `transform(X)` method available such as PCA/LDA from sklearn)
    :param centroids:       List of centroids learned in training phase
    :param centroids_label: Label corresponding to the centroids
    :param samples:         List of samples to recognize
    :return:                Predicted labels
    """

    pred = None 
    
    # YOUR CODE HERE
    data_centered = samples-samples.mean(axis=0)
    trsfrm.transform(samples).T # U = 
    U = trsfrm.components_[:m]
    #omega_n = np.array(np.split(U@train_centered, 38, axis=0))
    
    X = samples
    #X = X-X.mean(axis=0)
    mean = np.matmul(np.ones(X.shape[0]).reshape(X.shape[0],1),X.mean(0).reshape(1,X.shape[1]))
    X = X - mean

    omegas = U@X.T
    pred = []
    print(omegas.shape, X.shape)

    for om in omegas.T: # no .T
        dist = []
        for c in centroids:
            dist.append(np.linalg.norm(om-c))
        i = np.argmin(np.array(dist))
        """
        #dist = euclidean_distances(centroids.T, (om.reshape(-1, 1)@np.ones((1, centroids.shape[1]))))
        dist = euclidean_distances(centroids, om.reshape(-1, 1).T)
        print(dist)
        print(dist.shape, centroids.shape, om.reshape(-1, 1).T.shape)
        i = np.argmin(dist, axis=1)
        """
        pred.append(centroids_label[i])

    return pred

The next cell validates the implementation with the whole system on the training set. The expected recognition accuracy should by close to 100% depending on the task difficulty.

In [ ]:
# Recognize training set
pca_train_pred = recognize(pca, train_centroids, train_centroid_label, train_img)

# Compute performance
n_err = np.count_nonzero(np.where(pca_train_pred != train_label))
train_acc = 1.0 - n_err / train_label.size
print("The recognition accuracy on the trainig set is {:.2f}".format(train_acc))


**QUESTIONS** (/2)

In the nex cell, estimate the recognition accuracy on the testing set. Comment the results.


In [ ]:
test_acc = 0

# YOUR CODE HERE
test_img, test_label = load_images(test)
test_centered = test_img-test_img.mean(axis=0)
pca_test_pred = recognize(pca, train_centroids, train_centroid_label, test_img)
n_err = np.count_nonzero(np.where(pca_test_pred != test_label))
test_acc = 1.0 - n_err / test_label.size

print("The recognition accuracy on the testing set is {:.2f}".format(test_acc))

**YOUR ANSWER**

We can observe that the recognition accuracy on the `testing set` is **lower** than on the `training one`, but remains a good performance. This is what we should expect as we try to recognize new images that may be different from the training ones that were used to compute the centroids.

Measuring accuracy of the system is a good indicator of the overall performance but does not indicate where the system is performing poorly. This can be quantified using **Confusion Matrix**. It describes the performance of classification model and shows how the system is confused for each samples in the training set.

Such representation can be computed using `sklearn.metrics.confusion_matrix`, details are provided [here](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

**QUESTION** (/2)

In the next cell, calculate and visualize the confusion matrix using the function `plot_confusion_matrix()` of the `utils.py` file. Comment the results.

In [ ]:
def plot_confusion_matrix(cm, 
                          classes,
                          normalize=False,
                          title='Confusion matrix',
                          axes=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    
    :param cm: Confusion matrix
    :param classes: Classes name
    :param normalize: Indicate if the confusion matrix need to be normalized
    :param title: Plot's title
    :param axes: Subplot on which to display the image
    :param cmap: Colormap to use
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    # Show cm
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    #fmt = '.2f' if normalize else 'd'
    #thresh = cm.max() / 2.
    #for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    #    plt.text(j, i, format(cm[i, j], fmt),
    #             horizontalalignment="center",
    #             color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
#########################
# YOUR CODE HERE
#########################

cm = confusion_matrix(pca_test_pred, test_label)
classes = np.unique(train_label)

# Plot confusion matrix
plt.figure(figsize=(12, 7))
plot_confusion_matrix(cm, classes, normalize=True)


**YOUR ANSWER**

It can be seen that this matrix is **not perfectly symmetric** and is **not perfectly diagonal** even though it is close to being so. The values on each row should add up to one and so whenever we have an non-diagonal element not equal to zero, the diagonal element on the same row will be less than one. Those non-zero non-diagonal elements represent the location where a missclassification occured. The `x axis` indicates the label that was assigned to a picture and the `y axis` the true label of the picture.

The diagonal elements represent the successful classifications as the predicted labels match the true ones.

If the test prediction accuracy were to be 100% all the diagonal elements should be set to one 1 and the rest to 0 as we will see in section `2.3.3 Fisherface`.

### 2.3.3 Fisherface

The subspace computed before with *PCA* was looking at directions where the variation in the data is maximum without paying attention to the class each data point belongs to. Therefore this approach is unsupervised. The major drawback is that the class separability is not garantee to be optimum. 

The approach of *Linear Discriminant Analysis* is to find a subspace where the variation is large (*i.e. similar to PCA*) but also to maximize the inter-class separability by taking into account each sample's label. The figure below shows an example:

<img src="../data/lda_example.png" alt="Drawing" style="width: 400px;"/>

Such subspace can be computed as follow:

- Compute the scatter matrices (*intra-classes* / *inter-classes*)
- Compute the eigenvectors / eigenvalues
- Select the $K_m$ largest eigenvalues and their corresponding eigenvectors

Given a set of samples $\boldsymbol{I}_0, \dots, \boldsymbol{I}_N$ and their corresponding labels $y_0, \dots, y_N$, the intra-class scatter matrix is computed as follow:

$$
\boldsymbol{S}_w = \sum_{i=1}^N \left(\boldsymbol{I}_i - \boldsymbol{\mu}_{y_i}\right) \left(\boldsymbol{I}_i - \boldsymbol{\mu}_{y_i}\right)^{\top}
$$

where $\boldsymbol{\mu}_{k}$ is the sample mean of the $k^{th}$ class.
Then the inter-class scatter matrix is defined as:

$$
\boldsymbol{S}_b = \sum_{k=1}^{m} n_k (\boldsymbol{\mu}_k - \boldsymbol{\mu})(\boldsymbol{\mu}_k - \boldsymbol{\mu})^{\top}
$$

where $m$ is the number of classes, $\boldsymbol{\mu}$ is the overall sample average and $n_k$ is the number of samples in the $k^{th}$ class.

Finally the subspace $\boldsymbol{W}$ can be computed by solving the following generalizeed eigenvalue problem:

$$
\boldsymbol{S}_b \boldsymbol{W} = \lambda \boldsymbol{S}_w \boldsymbol{w}
$$

Finally at most $m-1$ generalized eigenvectors are useful to discriminate between $m$ classes.

In practice, such decomposition can be computed using `sklearn.discriminant_analysis.LinearDiscriminantAnalysis`, more information available [here](http://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html).

**QUESTION** (/2)

In the next cell, compute the *LDA* subspace similar to what you have done before and display the first 8 basis (*i.e. Fisherface*).


In [ ]:
# YOUR CODE HERE
lda = LDA()
lda.fit(train_img, train_label)
fisherfaces = lda.coef_

for i in range(8):
    plt.subplot(2,4,i+1)
    plt.title("EigenFace #"+str(i))
    plt.imshow(fisherfaces[i, :].reshape(int(dim[0]/2), int(dim[1]/2)), cmap='gray') ### Eigenfaces instead of pc I think
    plt.axis('off')
plt.show()

**QUESTIONS** (/2)

In the next cell, visualize the subspace created by the *LDA* decomposition, similarly to the PCA method. 

What do you see? Is it better than before ? Comment the results.


In [ ]:
# YOUR CODE HERE

proj_train = lda.transform(train_img)

# Visualize 
# Colors for distinct individuals
colors = LabelEncoder().fit_transform(train_label.ravel())

plt.figure(figsize=(12, 7))
plt.scatter(proj_train[:, 2], proj_train[:, 3], c=colors)
plt.xlabel('Axis 2')
plt.ylabel('Axis 1')
plt.title('Trainset clusters')

**YOUR ANSWER**

In this case, the subspace is more sparsed in the sense that we can see clusters that are aggregated by colour.  About thirty clusters can be seen, so can deduce that each cluster corresponds to a subject. This result is better than what we had for PCA as we can find an inter-subject seperation more easily. 

**QUESTION** (/2)

In the next cell, compute the recognition accuracy on the *training*/*testing* set for the *LDA* recognizer. Comment the results.


In [ ]:
train_acc = 0
test_acc = 0

######################
# YOUR CODE HERE
train_acc = lda.score(train_img, train_label)
test_acc = lda.score(test_img, test_label)
######################

print("The recognition accuracy on the trainig set is {:.2f}".format(train_acc))

print("The recognition accuracy on the testing set is {:.2f}".format(test_acc))

**YOUR ANSWER**

**QUESTION** (/2)

Compute the *Confusion Matrix* and comment on the result you have


In [ ]:

######################
cm = confusion_matrix(lda.predict(test_img), test_label)
classes = np.unique(train_label) # or np.arange(38)
######################

# Plot confusion matrix
plt.figure(figsize=(12, 7))
plot_confusion_matrix(cm, classes, normalize=True)

**YOUR ANSWER**

In the case of LDA, the matrix is symmetric and seems to be a pure diagonal matrix. The diagonal elements are equal to one and the other components are 0. This behaviour is unlike the one of the confusion matrix for PCA. The latter was not exaclty symmetric and some non diagonal terms were set to values between 0.2, 0.4 and not all the diagonal terms were set to one.

This difference between PCA and LDA is also observable in the training and testing accuracy which is 1.00 for LDA whil it is lower for PCA and explains this difference on the confusion matrix. There are no off-diagonal elements because the classification error is 0%.

**QUESTION** (/2) 

You have implemented / tested two approaches for face recognition which one works the best and why ? What's are the pro/cons of each method ?


**YOUR ANSWER**

LDA seems to be the best method since it achieves an accuracy of 100% for both training and testing images. As we could observe with the projection of the data on the subspace of LDA and PCA, LDA achieve a much better seperability of the clusters and thus has a better performance.

However, LDA may also have a drawbacks. 
- While PCA is an unsupervised learning method, LDA takes the class lables into account. This means that this method cannot be used without labels

- This method finds a trade off between two factors. It tries to find a subspace where the variation is large but also aims to maximize the inter-class separability by taking into account each sample's label. This 

PCA's drawback is its accuracy that is in general lower than LDA in the case where we have access to labels. However, the pro of this method is that it can be used without labels whil LDA not as it is unsupervised method.